## sqlite prac ##


In [1]:
import sqlite3
import os 

db_name = "test_database2.db"

test_db_folder = r"..\test_db"

db_dir = os.path.join(test_db_folder, db_name)

def create_and_view_schema():
    # 1. Connect (This creates the file if it doesn't exist)
    conn = sqlite3.connect(db_dir)
    cursor = conn.cursor()

    # 2. Create a Table (Defining the Schema)
    # We create a dummy table called 'employees'
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS employees (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL,
            role TEXT,
            salary REAL,
            hire_date TEXT
        )
    """)
    conn.commit()
    print(f"Database '{db_name}' and table 'employees' created successfully.\n")

    # ---------------------------------------------------------
    # Method A: Get the raw CREATE statement (The "Code" view)
    # ---------------------------------------------------------
    print("--- Schema View (Raw SQL) ---")
    cursor.execute("SELECT sql FROM sqlite_master WHERE type='table' AND name='employees'")
    schema_raw = cursor.fetchone()
    
    if schema_raw:
        print(schema_raw[0])
    else:
        print("Table not found.")

    # ---------------------------------------------------------
    # Method B: Get the detailed column info (The "Table" view)
    # ---------------------------------------------------------
    print("\n--- Schema View (Column Details) ---")
    print(f"{'ID':<5} {'Name':<15} {'Type':<10} {'NotNull':<10} {'PK':<5}")
    print("-" * 50)
    
    # PRAGMA table_info returns: (cid, name, type, notnull, dflt_value, pk)
    cursor.execute(f"PRAGMA table_info(employees)")
    columns = cursor.fetchall()
    
    for col in columns:
        cid, name, dtype, notnull, dflt, pk = col
        print(f"{cid:<5} {name:<15} {dtype:<10} {notnull:<10} {pk:<5}")

    conn.close()

if __name__ == "__main__":
    create_and_view_schema()

Database 'test_database2.db' and table 'employees' created successfully.

--- Schema View (Raw SQL) ---
CREATE TABLE employees (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL,
            role TEXT,
            salary REAL,
            hire_date TEXT
        )

--- Schema View (Column Details) ---
ID    Name            Type       NotNull    PK   
--------------------------------------------------
0     id              INTEGER    0          1    
1     name            TEXT       1          0    
2     role            TEXT       0          0    
3     salary          REAL       0          0    
4     hire_date       TEXT       0          0    


## this is the sample code about getting the SP500 ticker value ##

In [1]:
import pandas as pd
import yfinance as yf
import sqlite3
import datetime
import time

def get_sp500_tickers():
    """Retreives the current S&P 500 tickers from Wikipedia."""
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    try:
        tables = pd.read_html(url)
        df = tables[0]
        # Wikipedia uses dots (BRK.B) but Yahoo uses dashes (BRK-B)
        tickers = df['Symbol'].str.replace('.', '-', regex=False).tolist()
        return tickers
    except Exception as e:
        print(f"Error fetching tickers: {e}")
        return []

def save_to_sqlite(data, db_name="sp500_data.db", table_name="stock_history"):
    """Saves a pandas DataFrame to a SQLite database."""
    try:
        conn = sqlite3.connect(db_name)
        # 'if_exists="append"' allows adding data in chunks
        data.to_sql(table_name, conn, if_exists="append", index=True)
        conn.close()
        print(f"Successfully saved {len(data)} rows to {db_name}.")
    except Exception as e:
        print(f"Error saving to database: {e}")


# 1. Setup
start_date = "2001-01-01"
end_date = datetime.date.today().strftime("%Y-%m-%d")
db_name = "sp500_data.db"

# 2. Get Tickers
print("Fetching S&P 500 tickers from Wikipedia...")
tickers = get_sp500_tickers()
if not tickers:
    print("No tickers found. Exiting.")
    exit(1)
print(f"Found {len(tickers)} tickers.")

# 3. Download and Store in Batches
# Downloading 500+ tickers at once for 20+ years can cause timeouts or memory issues.
# We will process them in batches of 20.
batch_size = 20

# Initialize database (optional: clear old data if you want a fresh start)
# open(db_name, 'w').close() 

for i in range(0, len(tickers), batch_size):
    batch = tickers[i:i + batch_size]
    print(f"Processing batch {i // batch_size + 1}/{(len(tickers) // batch_size) + 1} ({batch[0]} - {batch[-1]})...")
    
    try:
        # group_by='ticker' ensures the DataFrame is structured correctly for stacking
        data = yf.download(batch, start=start_date, end=end_date, group_by='ticker', progress=False, threads=True)
        
        # yfinance returns a multi-level column DataFrame (Ticker -> OHLCV).
        # We need to reshape it to be database-friendly (Date, Ticker, Open, Close, etc.)
        
        # Stack the ticker level to create a 'Ticker' column
        data = data.stack(level=0, future_stack=True)
        
        # Reset index to make 'Date' and 'Ticker' regular columns
        data.reset_index(inplace=True)
        
        # Rename columns to be SQL friendly
        data.columns.name = None # Remove the name of the columns index
        data = data.rename(columns={
            'Date': 'date', 
            'Ticker': 'ticker', 
            'Open': 'open', 
            'High': 'high', 
            'Low': 'low', 
            'Close': 'close', 
            'Adj Close': 'adj_close', 
            'Volume': 'volume'
        })
        
        # Save this batch to DB
        save_to_sqlite(data, db_name)
        
        # Sleep slightly to be polite to the API
        time.sleep(1)
        
    except Exception as e:
        print(f"Failed to process batch starting with {batch[0]}: {e}")

print("Data retrieval complete.")



Fetching S&P 500 tickers from Wikipedia...
Error fetching tickers: HTTP Error 403: Forbidden
No tickers found. Exiting.
Found 0 tickers.
Data retrieval complete.


In [ ]:
import pandas as pd
import yfinance as yf
import sqlite3
import datetime

tickers = [ "SPY" ,"NVDA", "AAPL", "MSFT", "AMZN" \
           ,"GOOGL", "META", "TSLA", "BRK-B", "JPM" \
            ,"AVGO", "INTC", "WMT", "UNH", "V", "PYPL" \
            ,"MA", "HD", "BAC", "COST", "DIS" ] 

def get_ticker(tickers):
    """
    Retrieves historical data specifically for the SPY ETF (SPDR S&P 500 ETF Trust)
    and saves it to the database.
    """
    ticker = "SPY"
    start_date = "2001-01-01"
    end_date = datetime.date.today().strftime("%Y-%m-%d")
    db_name = "sp500_data.db"
    
    print(f"Fetching data for {ticker} from {start_date} to today...")

    try:
        # 1. Download Data
        # auto_adjust=False ensures we get both 'Close' and 'Adj Close' columns
        data = yf.download(ticker, start=start_date, end=end_date, progress=False, auto_adjust=False)
        
        if data.empty:
            print("No data found for SPY.")
            return

        # 2. Format Data
        # Reset index to turn the 'Date' index into a standard column
        data.reset_index(inplace=True)
        
        # Add a 'ticker' column explicitly so we know these rows belong to SPY
        data['ticker'] = ticker

        # Rename columns to be lowercase and SQL-friendly
        data = data.rename(columns={
            'Date': 'date', 
            'Open': 'open', 
            'High': 'high', 
            'Low': 'low', 
            'Close': 'close', 
            'Adj Close': 'adj_close', 
            'Volume': 'volume'
        })

        # Select and reorder columns to ensure they match the database schema
        # Note: If yfinance returns multi-level columns, we flatten them or select specific ones
        # This check handles occasional formatting differences in yfinance versions
        if isinstance(data.columns, pd.MultiIndex):
            data.columns = data.columns.get_level_values(0)

        final_columns = ['date', 'ticker', 'open', 'high', 'low', 'close', 'adj_close', 'volume']
        data = data[final_columns]

        # 3. Save to Database
        conn = sqlite3.connect(db_name)
        data.to_sql("stock_history", conn, if_exists="append", index=False)
        conn.close()
        
        print(f"Successfully saved {len(data)} rows for {ticker} to {db_name}.")
        
    except Exception as e:
        print(f"Error fetching {ticker}: {e}")

# --- Test the function ---
if __name__ == "__main__":
    get_spy_ticker()

Fetching data for SPY from 2001-01-01 to today...
Successfully saved 6280 rows for SPY to sp500_data.db.


In [5]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("sp500_data.db")

# Example: Get all Apple data
df = pd.read_sql("SELECT * FROM stock_history WHERE ticker = 'SPY'", conn)
print(df.head())

# Example: Get all closing prices for a specific date
df_date = pd.read_sql("SELECT ticker, close FROM stock_history WHERE date = '2023-01-04 00:00:00'", conn)

conn.close()

                  date ticker       open       high        low       close  \
0  2001-01-02 00:00:00    SPY  132.00000  132.15625  127.56250  128.812500   
1  2001-01-03 00:00:00    SPY  128.31250  136.00000  127.65625  135.000000   
2  2001-01-04 00:00:00    SPY  134.93750  135.46875  133.00000  133.546875   
3  2001-01-05 00:00:00    SPY  133.46875  133.62500  129.18750  129.187500   
4  2001-01-08 00:00:00    SPY  129.87500  130.18750  127.68750  130.187500   

   adj_close    volume  
0  81.997665   8737500  
1  85.936356  19431600  
2  85.011391   9219000  
3  82.236374  12911400  
4  82.872902   6625300  


In [9]:
df

,date,ticker,open,high,low,close,adj_close,volume
0,2001-01-02 00:00:00,SPY,132.000000,132.156250,127.562500,128.812500,81.997665,8737500
1,2001-01-03 00:00:00,SPY,128.312500,136.000000,127.656250,135.000000,85.936356,19431600
2,2001-01-04 00:00:00,SPY,134.937500,135.468750,133.000000,133.546875,85.011391,9219000
3,2001-01-05 00:00:00,SPY,133.468750,133.625000,129.187500,129.187500,82.236374,12911400
4,2001-01-08 00:00:00,SPY,129.875000,130.187500,127.687500,130.187500,82.872902,6625300
...,...,...,...,...,...,...,...,...
6275,2025-12-15 00:00:00,SPY,685.739990,685.760010,679.250000,680.729980,678.724426,90811000
6276,2025-12-16 00:00:00,SPY,679.229980,681.080017,674.979980,678.869995,676.869934,122030600
6277,2025-12-17 00:00:00,SPY,679.890015,680.440002,671.200012,671.400024,669.421936,110625200
6278,2025-12-18 00:00:00,SPY,677.599976,680.739990,674.900024,676.469971,674.476929,108650100


## get all the ticket and data ##

In [2]:
import pandas as pd
import yfinance as yf
import sqlite3
import datetime

tickers = [ "SPY" ,"NVDA", "AAPL", "MSFT", "AMZN" \
           ,"GOOGL", "META", "TSLA", "BRK-B", "JPM" \
            ,"AVGO", "INTC", "WMT", "UNH", "V", "PYPL" \
            ,"MA", "HD", "BAC", "COST", "DIS" ] 

def get_ticker(tickers):
    """
    Retrieves historical data specifically for the SPY ETF (SPDR S&P 500 ETF Trust)
    and saves it to the database.
    """
    for ticker in tickers:
        start_date = "2001-01-01"
        end_date = datetime.date.today().strftime("%Y-%m-%d")
        db_name = f"{ticker}_data.db"
        
        print(f"Fetching data for {ticker} from {start_date} to today...")

        try:
            # 1. Download Data
            # auto_adjust=False ensures we get both 'Close' and 'Adj Close' columns
            data = yf.download(ticker, start=start_date, end=end_date, progress=False, auto_adjust=False)
            
            if data.empty:
                print("No data found for SPY.")
                return

            # 2. Format Data
            # Reset index to turn the 'Date' index into a standard column
            data.reset_index(inplace=True)
            
            # Add a 'ticker' column explicitly so we know these rows belong to SPY
            data['ticker'] = ticker

            # Rename columns to be lowercase and SQL-friendly
            data = data.rename(columns={
                'Date': 'date', 
                'Open': 'open', 
                'High': 'high', 
                'Low': 'low', 
                'Close': 'close', 
                'Adj Close': 'adj_close', 
                'Volume': 'volume'})

            # Select and reorder columns to ensure they match the database schema
            # Note: If yfinance returns multi-level columns, we flatten them or select specific ones
            # This check handles occasional formatting differences in yfinance versions
            if isinstance(data.columns, pd.MultiIndex):
                data.columns = data.columns.get_level_values(0)

            final_columns = ['date', 'ticker', 'open', 'high', 'low', 'close', 'adj_close', 'volume']
            data = data[final_columns]

            # 3. Save to Database
            conn = sqlite3.connect(db_name)
            data.to_sql("stock_history", conn, if_exists="append", index=False)
            conn.close()
            
            print(f"Successfully saved {len(data)} rows for {ticker} to {db_name}.")
        
        except Exception as e:
            print(f"Error fetching {ticker}: {e}")

tickers = [ "SPY" ,"NVDA", "AAPL", "MSFT", "AMZN" \
           ,"GOOGL", "META", "TSLA", "BRK-B", "JPM" \
            ,"AVGO", "INTC", "WMT", "UNH", "V", "PYPL" \
            ,"MA", "HD", "BAC", "COST", "DIS" ] 
get_ticker(tickers)

Fetching data for SPY from 2001-01-01 to today...
Successfully saved 6288 rows for SPY to SPY_data.db.
Fetching data for NVDA from 2001-01-01 to today...
Successfully saved 6288 rows for NVDA to NVDA_data.db.
Fetching data for AAPL from 2001-01-01 to today...
Successfully saved 6288 rows for AAPL to AAPL_data.db.
Fetching data for MSFT from 2001-01-01 to today...
Successfully saved 6288 rows for MSFT to MSFT_data.db.
Fetching data for AMZN from 2001-01-01 to today...
Successfully saved 6288 rows for AMZN to AMZN_data.db.
Fetching data for GOOGL from 2001-01-01 to today...
Successfully saved 5378 rows for GOOGL to GOOGL_data.db.
Fetching data for META from 2001-01-01 to today...
Successfully saved 3426 rows for META to META_data.db.
Fetching data for TSLA from 2001-01-01 to today...
Successfully saved 3903 rows for TSLA to TSLA_data.db.
Fetching data for BRK-B from 2001-01-01 to today...
Successfully saved 6288 rows for BRK-B to BRK-B_data.db.
Fetching data for JPM from 2001-01-01 to to

In [1]:
# ******** this is the production version to get the close and open prices and save to a specific folder ********

import pandas as pd
import yfinance as yf
import sqlite3
import datetime
import os  # <--- Import this to handle paths

tickers = [ "SPY" ,"NVDA", "AAPL", "MSFT", "AMZN" \
           ,"GOOGL", "META", "TSLA", "BRK-B", "JPM" \
            ,"AVGO", "INTC", "WMT", "UNH", "V", "PYPL" \
            ,"MA", "HD", "BAC", "COST", "DIS", "TSM", \
          "ARM", "BABA"] 

def get_ticker(tickers):
    
    # --- CONFIGURATION ---
    # 1. Define the specific folder path
    # Windows Example: r"C:\Users\YourName\Documents\FinanceData"
    # Mac/Linux Example: "/Users/yourname/Documents/FinanceData"
    # The 'r' before the string handles backslashes safely in Windows
    target_folder = r"C:\Users\admin\Desktop\R\Projects\Stock\stock_daily_data"
    
    # 2. Define the file name
    db_filename = "market_data_master.db"
    
    # 3. Create the full path safely
    db_path = os.path.join(target_folder, db_filename)

    # 4. Check if directory exists, if not, create it automatically
    if not os.path.exists(target_folder):
        os.makedirs(target_folder)
        print(f"Created directory: {target_folder}")

    print(f"Database will be saved to: {db_path}")

    # --- MAIN LOOP ---
    start_date = "2001-01-01"
    end_date = datetime.date.today().strftime("%Y-%m-%d")

    for ticker in tickers:
        print(f"Fetching data for {ticker}...")

        try:
            data = yf.download(ticker, start=start_date, end=end_date, progress=False, auto_adjust=False)
            
            if data.empty:
                continue

            data.reset_index(inplace=True)
            data['ticker'] = ticker

            data = data.rename(columns={
                'Date': 'date', 'Open': 'open', 'High': 'high', 
                'Low': 'low', 'Close': 'close', 'Adj Close': 'adj_close', 
                'Volume': 'volume'
            })

            if isinstance(data.columns, pd.MultiIndex):
                data.columns = data.columns.get_level_values(0)

            final_columns = ['date', 'ticker', 'open', 'high', 'low', 'close', 'adj_close', 'volume']
            data = data[final_columns]

            # --- CONNECT USING THE FULL PATH ---
            conn = sqlite3.connect(db_path)  # <--- Use db_path here
            
            table_name = ticker.replace("-", "_")
            data.to_sql(table_name, conn, if_exists="replace", index=False)
            conn.close()
            
            print(f"Saved {ticker} to {db_path}")
        
        except Exception as e:
            print(f"Error fetching {ticker}: {e}")

if __name__ == "__main__":
    get_ticker(tickers)

Database will be saved to: C:\Users\admin\Desktop\R\Projects\Stock\stock_daily_data\market_data_master.db
Fetching data for SPY...
Saved SPY to C:\Users\admin\Desktop\R\Projects\Stock\stock_daily_data\market_data_master.db
Fetching data for NVDA...
Saved NVDA to C:\Users\admin\Desktop\R\Projects\Stock\stock_daily_data\market_data_master.db
Fetching data for AAPL...
Saved AAPL to C:\Users\admin\Desktop\R\Projects\Stock\stock_daily_data\market_data_master.db
Fetching data for MSFT...
Saved MSFT to C:\Users\admin\Desktop\R\Projects\Stock\stock_daily_data\market_data_master.db
Fetching data for AMZN...
Saved AMZN to C:\Users\admin\Desktop\R\Projects\Stock\stock_daily_data\market_data_master.db
Fetching data for GOOGL...
Saved GOOGL to C:\Users\admin\Desktop\R\Projects\Stock\stock_daily_data\market_data_master.db
Fetching data for META...
Saved META to C:\Users\admin\Desktop\R\Projects\Stock\stock_daily_data\market_data_master.db
Fetching data for TSLA...
Saved TSLA to C:\Users\admin\Deskto

In [ ]:
# test version for fundamentals and history

import pandas as pd
import yfinance as yf
import sqlite3
import datetime

# 1. The Mapping you created (API Keys -> Your Database Columns)
FUNDAMENTAL_MAPPING = {
    'last_price': 'last',
    'bid': 'bid',
    'ask': 'ask',
    'bidSize': 'bid_size',
    'askSize': 'ask_size',
    'currentPrice': 'current_price',
    'targetHighPrice': 'target_high_price',
    'targetLowPrice': 'target_low_price',
    'targetMeanPrice': 'target_mean_price',
    'targetMedianPrice': 'target_median_price',
    'recommendationKey': 'recommendation_key',
    'numberOfAnalystOpinions': 'number_of_analyst_opinions',
    'lastDividendValue': 'last_dividend_value',
    'lastDividendDate': 'last_dividend_date',
    'totalCash': 'total_cash',
    'totalCashPerShare': 'total_cash_per_share',
    'ebitda': 'ebitda',
    'totalDebt': 'total_debt',
    'quickRatio': 'quick_ratio',
    'currentRatio': 'current_ratio',
    'totalRevenue': 'total_revenue',
    'debtToEquity': 'debt_to_equity',
    'revenuePerShare': 'revenue_per_share',
    'returnOnAssets': 'return_on_assets',
    'returnOnEquity': 'return_on_equity',
    'grossProfits': 'gross_profits',
    'freeCashflow': 'free_cashflow',
    'operatingCashflow': 'operating_cashflow',
    'earningsGrowth': 'earnings_growth',
    'revenueGrowth': 'revenue_growth',
    'grossMargins': 'gross_margins',
    'ebitdaMargins': 'ebitda_margins',
    'operatingMargins': 'operating_margins',
    'regularMarketChange': 'regular_market_change',
    'regularMarketChangePercent': 'regular_market_change_percent',
    'regularMarketPrice': 'regular_market_price',
    'postMarketChangePercent': 'post_market_change_percent',
    'postMarketPrice': 'post_market_price',
    'postMarketChange': 'post_market_change',
    'averageDailyVolume3Month': 'average_daily_volume_3_month',
    'open': 'open_price',
    'dayLow': 'day_low',
    'dayHigh': 'day_high',
    'regularMarketPreviousClose': 'regular_market_previous_close',
    'regularMarketOpen': 'regular_market_open',
    'regularMarketDayLow': 'regular_market_day_low',
    'regularMarketDayHigh': 'regular_market_day_high',
    'dividendRate': 'dividend_rate',
    'dividendYield': 'dividend_yield',
    'trailingPE': 'trail_PE',
    'forwardPE': 'forward_PE',
    'trailingEps': 'trail_Eps',
    'forwardEps': 'forward_Eps',
    'previous_close': 'prev_close',
    'volume': 'volume',
    'regularMarketVolume': 'reg_market_volume',
    'averageVolume': 'avg_volume',
    'averageVolume10days': 'ave_10d_volume',
    'currency': 'currency',
    'exchange': 'exchange',
    'profitMargins': 'profit_margin',
    'floatShares': 'float_shares',
    'enterpriseValue': 'company_value',
    'sharesOutstanding': 'outstanding_share',
    'sharesShort': 'share_short',
    'sharesPercentSharesOut': 'share_percent_shares_out',
    'heldPercentInsiders': 'insider_held_percent',
    'heldPercentInstitutions': 'institution_held_percent',
    'shortRatio': 'short_ratio',
    'shortPercentOfFloat': 'short_percent_of_float',
    'impliedSharesOutstanding': 'implied_shares_outstanding',
    'bookValue': 'book_value',
    'priceToBook': 'price_2_book'
}

def save_historical_prices(ticker, db_name="sp500_data.db"):
    """Downloads OHLCV history (2001-Today) and saves to 'stock_history' table."""
    start_date = "2001-01-01"
    print(f"[{ticker}] Fetching Historical Prices...")
    
    try:
        # Fetch history
        df = yf.download(ticker, start=start_date, progress=False, auto_adjust=False)
        
        if df.empty:
            print(f"[{ticker}] No price data found.")
            return

        # Formatting
        df.reset_index(inplace=True)
        df['ticker'] = ticker
        
        # Rename standard columns to lower case
        df.rename(columns={
            'Date': 'date', 'Open': 'open', 'High': 'high', 
            'Low': 'low', 'Close': 'close', 'Adj Close': 'adj_close', 
            'Volume': 'volume'
        }, inplace=True)

        # Handle MultiIndex if present (common issue with new yfinance versions)
        if isinstance(df.columns, pd.MultiIndex):
            df.columns = df.columns.get_level_values(0)

        # Keep only valid columns
        valid_cols = ['date', 'ticker', 'open', 'high', 'low', 'close', 'adj_close', 'volume']
        # Filter for columns that actually exist in the dataframe
        cols_to_save = [c for c in valid_cols if c in df.columns]
        df = df[cols_to_save]

        # Save
        conn = sqlite3.connect(db_name)
        df.to_sql("stock_history", conn, if_exists="append", index=False)
        conn.close()
        print(f"[{ticker}] Saved {len(df)} days of price history.")

    except Exception as e:
        print(f"[{ticker}] Error saving history: {e}")

def save_fundamentals(ticker, db_name="sp500_data.db"):
    """Fetches CURRENT fundamental snapshot and saves to 'stock_fundamentals' table."""
    print(f"[{ticker}] Fetching Fundamentals...")
    
    try:
        # Fetch info dictionary
        ticker_obj = yf.Ticker(ticker)
        info = ticker_obj.info
        
        # Extract data using your MAPPING
        data_row = {'ticker': ticker, 'date_scraped': datetime.date.today()}
        
        for api_key, my_col_name in FUNDAMENTAL_MAPPING.items():
            # info.get(key, None) safely returns None if data is missing
            data_row[my_col_name] = info.get(api_key, None)

        # Convert single row dictionary to DataFrame
        df = pd.DataFrame([data_row])
        
        # Save
        conn = sqlite3.connect(db_name)
        df.to_sql("stock_fundamentals", conn, if_exists="append", index=False)
        conn.close()
        print(f"[{ticker}] Saved fundamental data.")

    except Exception as e:
        print(f"[{ticker}] Error saving fundamentals: {e}")

def process_stock(ticker):
    # 1. Save History (The 2001-Today Chart Data)
    save_historical_prices(ticker)
    
    # 2. Save Fundamentals (The specific list of columns you asked for)
    save_fundamentals(ticker)

if __name__ == "__main__":
    # You can change this to "SPY", "NVDA", or run a loop for all S&P 500
    process_stock("SPY")

[SPY] Fetching Historical Prices...
[SPY] Saved 6288 days of price history.
[SPY] Fetching Fundamentals...
[SPY] Saved fundamental data.


In [1]:
import pandas as pd
import yfinance as yf
import sqlite3
import datetime

# 1. Define the Mapping (API Keys -> Your Column Names)
FUNDAMENTAL_MAPPING = {
    'last_price': 'last',
    'bid': 'bid',
    'ask': 'ask',
    'bidSize': 'bid_size',
    'askSize': 'ask_size',
    'currentPrice': 'current_price',
    'targetHighPrice': 'target_high_price',
    'targetLowPrice': 'target_low_price',
    'targetMeanPrice': 'target_mean_price',
    'targetMedianPrice': 'target_median_price',
    'recommendationKey': 'recommendation_key',
    'numberOfAnalystOpinions': 'number_of_analyst_opinions',
    'lastDividendValue': 'last_dividend_value',
    'lastDividendDate': 'last_dividend_date',
    'totalCash': 'total_cash',
    'totalCashPerShare': 'total_cash_per_share',
    'ebitda': 'ebitda',
    'totalDebt': 'total_debt',
    'quickRatio': 'quick_ratio',
    'currentRatio': 'current_ratio',
    'totalRevenue': 'total_revenue',
    'debtToEquity': 'debt_to_equity',
    'revenuePerShare': 'revenue_per_share',
    'returnOnAssets': 'return_on_assets',
    'returnOnEquity': 'return_on_equity',
    'grossProfits': 'gross_profits',
    'freeCashflow': 'free_cashflow',
    'operatingCashflow': 'operating_cashflow',
    'earningsGrowth': 'earnings_growth',
    'revenueGrowth': 'revenue_growth',
    'grossMargins': 'gross_margins',
    'ebitdaMargins': 'ebitda_margins',
    'operatingMargins': 'operating_margins',
    'regularMarketChange': 'regular_market_change',
    'regularMarketChangePercent': 'regular_market_change_percent',
    'regularMarketPrice': 'regular_market_price',
    'postMarketChangePercent': 'post_market_change_percent',
    'postMarketPrice': 'post_market_price',
    'postMarketChange': 'post_market_change',
    'averageDailyVolume3Month': 'average_daily_volume_3_month',
    'open': 'open_price',
    'dayLow': 'day_low',
    'dayHigh': 'day_high',
    'regularMarketPreviousClose': 'regular_market_previous_close',
    'regularMarketOpen': 'regular_market_open',
    'regularMarketDayLow': 'regular_market_day_low',
    'regularMarketDayHigh': 'regular_market_day_high',
    'dividendRate': 'dividend_rate',
    'dividendYield': 'dividend_yield',
    'trailingPE': 'trail_PE',
    'forwardPE': 'forward_PE',
    'trailingEps': 'trail_Eps',
    'forwardEps': 'forward_Eps',
    'previous_close': 'prev_close',
    'volume': 'volume',
    'regularMarketVolume': 'reg_market_volume',
    'averageVolume': 'avg_volume',
    'averageVolume10days': 'ave_10d_volume',
    'currency': 'currency',
    'exchange': 'exchange',
    'profitMargins': 'profit_margin',
    'floatShares': 'float_shares',
    'enterpriseValue': 'company_value',
    'sharesOutstanding': 'outstanding_share',
    'sharesShort': 'share_short',
    'sharesPercentSharesOut': 'share_percent_shares_out',
    'heldPercentInsiders': 'insider_held_percent',
    'heldPercentInstitutions': 'institution_held_percent',
    'shortRatio': 'short_ratio',
    'shortPercentOfFloat': 'short_percent_of_float',
    'impliedSharesOutstanding': 'implied_shares_outstanding',
    'bookValue': 'book_value',
    'priceToBook': 'price_2_book'
}

def get_fundamental_data(ticker, db_name="sp500_data.db"):
    """
    Fetches the current fundamental snapshot for a ticker and saves it to the database.
    """
    print(f"[{ticker}] Fetching fundamental data...")

    try:
        # 1. Fetch Data
        stock = yf.Ticker(ticker)
        # .info is where the fundamental dictionary lives
        info_data = stock.info
        
        # 2. Extract Data using Mapping
        # We start with ticker and date since fundamentals are a snapshot in time
        row_data = {
            'ticker': ticker,
            'time_scraped': datetime.datetime.now()
        }

        for api_key, db_column in FUNDAMENTAL_MAPPING.items():
            # .get() prevents crashing if a specific key is missing from Yahoo
            row_data[db_column] = info_data.get(api_key, None)
            
            # Fallback logic for keys that might have alternate names (like EPS)
            if db_column == 'trail_Eps' and row_data[db_column] is None:
                row_data[db_column] = info_data.get('epsTrailing', None)
            if db_column == 'forward_Eps' and row_data[db_column] is None:
                row_data[db_column] = info_data.get('epsForward', None)
            if db_column == 'volume' and row_data[db_column] is None:
                row_data[db_column] = info_data.get('last_volume', None)

        # 3. Create DataFrame
        df = pd.DataFrame([row_data])

        # 4. Save to Database
        conn = sqlite3.connect(db_name)
        # We use a DIFFERENT table name 'stock_fundamentals' to avoid mixing with daily price history
        df.to_sql("stock_fundamentals", conn, if_exists="append", index=False)
        conn.close()

        print(f"[{ticker}] Successfully saved fundamentals.")

    except Exception as e:
        print(f"[{ticker}] Error: {e}")

# --- Execution ---
if __name__ == "__main__":
    # Example: Get fundamentals for SPY and NVDA
    tickers_to_fetch = ["SPY", "NVDA", "AAPL"]
    
    for symbol in tickers_to_fetch:
        get_fundamental_data(symbol)

[SPY] Fetching fundamental data...
[SPY] Successfully saved fundamentals.
[NVDA] Fetching fundamental data...
[NVDA] Successfully saved fundamentals.
[AAPL] Fetching fundamental data...
[AAPL] Successfully saved fundamentals.


In [ ]:
# this is the scheduled version 
# **** this is production version at the mini server to get fundamentals daily ****

import pandas as pd
import yfinance as yf
import sqlite3
import datetime
import schedule
import time

# --- CONFIGURATION ---
DB_NAME = "market_data.db"
TICKERS = [ "SPY" ,"NVDA", "AAPL", "MSFT", "AMZN" \
           ,"GOOGL", "META", "TSLA", "BRK-B", "JPM" \
            ,"AVGO", "INTC", "WMT", "UNH", "V", "PYPL" \
            ,"MA", "HD", "BAC", "COST", "DIS" ] 
MARKET_OPEN = datetime.time(9, 30)
MARKET_CLOSE = datetime.time(16, 0)

# 1. Define the Mapping (kept exactly as you had it)
FUNDAMENTAL_MAPPING = {
    'last_price': 'last',
    'bid': 'bid',
    'ask': 'ask',
    'bidSize': 'bid_size',
    'askSize': 'ask_size',
    'currentPrice': 'current_price',
    'targetHighPrice': 'target_high_price',
    'targetLowPrice': 'target_low_price',
    'targetMeanPrice': 'target_mean_price',
    'targetMedianPrice': 'target_median_price',
    'recommendationKey': 'recommendation_key',
    'numberOfAnalystOpinions': 'number_of_analyst_opinions',
    'lastDividendValue': 'last_dividend_value',
    'lastDividendDate': 'last_dividend_date',
    'totalCash': 'total_cash',
    'totalCashPerShare': 'total_cash_per_share',
    'ebitda': 'ebitda',
    'totalDebt': 'total_debt',
    'quickRatio': 'quick_ratio',
    'currentRatio': 'current_ratio',
    'totalRevenue': 'total_revenue',
    'debtToEquity': 'debt_to_equity',
    'revenuePerShare': 'revenue_per_share',
    'returnOnAssets': 'return_on_assets',
    'returnOnEquity': 'return_on_equity',
    'grossProfits': 'gross_profits',
    'freeCashflow': 'free_cashflow',
    'operatingCashflow': 'operating_cashflow',
    'earningsGrowth': 'earnings_growth',
    'revenueGrowth': 'revenue_growth',
    'grossMargins': 'gross_margins',
    'ebitdaMargins': 'ebitda_margins',
    'operatingMargins': 'operating_margins',
    'regularMarketChange': 'regular_market_change',
    'regularMarketChangePercent': 'regular_market_change_percent',
    'regularMarketPrice': 'regular_market_price',
    'postMarketChangePercent': 'post_market_change_percent',
    'postMarketPrice': 'post_market_price',
    'postMarketChange': 'post_market_change',
    'averageDailyVolume3Month': 'average_daily_volume_3_month',
    'open': 'open_price',
    'dayLow': 'day_low',
    'dayHigh': 'day_high',
    'regularMarketPreviousClose': 'regular_market_previous_close',
    'regularMarketOpen': 'regular_market_open',
    'regularMarketDayLow': 'regular_market_day_low',
    'regularMarketDayHigh': 'regular_market_day_high',
    'dividendRate': 'dividend_rate',
    'dividendYield': 'dividend_yield',
    'trailingPE': 'trail_PE',
    'forwardPE': 'forward_PE',
    'trailingEps': 'trail_Eps',
    'forwardEps': 'forward_Eps',
    'previous_close': 'prev_close',
    'volume': 'volume',
    'regularMarketVolume': 'reg_market_volume',
    'averageVolume': 'avg_volume',
    'averageVolume10days': 'ave_10d_volume',
    'currency': 'currency',
    'exchange': 'exchange',
    'profitMargins': 'profit_margin',
    'floatShares': 'float_shares',
    'enterpriseValue': 'company_value',
    'sharesOutstanding': 'outstanding_share',
    'sharesShort': 'share_short',
    'sharesPercentSharesOut': 'share_percent_shares_out',
    'heldPercentInsiders': 'insider_held_percent',
    'heldPercentInstitutions': 'institution_held_percent',
    'shortRatio': 'short_ratio',
    'shortPercentOfFloat': 'short_percent_of_float',
    'impliedSharesOutstanding': 'implied_shares_outstanding',
    'bookValue': 'book_value',
    'priceToBook': 'price_2_book'
}

def get_fundamental_data(ticker):
    """
    Fetches the current fundamental snapshot for a ticker and saves it to the database.
    """
    print(f"[{ticker}] Fetching fundamental data...")

    try:
        stock = yf.Ticker(ticker)
        info_data = stock.info
        
        row_data = {
            'ticker': ticker,
            'time_scraped': datetime.datetime.now()
        }

        for api_key, db_column in FUNDAMENTAL_MAPPING.items():
            row_data[db_column] = info_data.get(api_key, None)
            
            # Fallback logic
            if db_column == 'trail_Eps' and row_data[db_column] is None:
                row_data[db_column] = info_data.get('epsTrailing', None)
            if db_column == 'forward_Eps' and row_data[db_column] is None:
                row_data[db_column] = info_data.get('epsForward', None)
            if db_column == 'volume' and row_data[db_column] is None:
                row_data[db_column] = info_data.get('last_volume', None)

        df = pd.DataFrame([row_data])

        conn = sqlite3.connect(DB_NAME)
        # Using 'append' so every 20 mins adds a new snapshot (Time Series data)
        df.to_sql("stock_fundamentals", conn, if_exists="append", index=False)
        conn.close()

        print(f"[{ticker}] Successfully saved fundamentals.")

    except Exception as e:
        print(f"[{ticker}] Error: {e}")

def job():
    """
    This function checks if the current time is within market hours.
    If yes, it runs the scraper.
    """
    now = datetime.datetime.now().time()
    # Check if within 9:30 AM to 4:00 PM
    if MARKET_OPEN <= now <= MARKET_CLOSE:
        print(f"\n--- Starting Scheduled Run: {datetime.datetime.now()} ---")
        for symbol in TICKERS:
            get_fundamental_data(symbol)
        print("--- Run Complete ---\n")
    else:
        print(f"Market Closed. Current time: {now}. Waiting...", end='\r')

# --- SCHEDULING LOGIC ---
if __name__ == "__main__":
    print("Scheduler Started. Waiting for market hours (Mon-Fri, 9:30-16:00)...")
    
    # 1. Schedule the job to run every 20 minutes
    schedule.every(10).minutes.do(job)

    # Define the specific exit time
    EXIT_TIME = datetime.time(16, 0)

    # 2. Infinite loop to keep the script running
    while True:
        now = datetime.datetime.now()
        
        # --- EXIT CHECK ---
        # If the current time is past 5:00 PM, break the loop to exit
        if now.time() >= EXIT_TIME:
            print(f"Current time is {now.strftime('%H:%M')}. Past 5:00 PM. Exiting program.")
            break

        # Check if today is a weekday (0=Mon, 4=Fri)
        if now.weekday() < 5:
            schedule.run_pending()
        else:
            # Optional: Pass on weekends
            pass
            
        # Check every 60 seconds (better accuracy than 500s)
        time.sleep(60)

In [14]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("sp500_data.db")

# Example: Get all Apple data
df = pd.read_sql("SELECT * FROM stock_history WHERE ticker = 'SPY'", conn)
print(df.head())

# Example: Get all closing prices for a specific date
df_date = pd.read_sql("SELECT ticker, close FROM stock_history WHERE date = '2023-01-04 00:00:00'", conn)

conn.close()

                  date ticker       open       high        low       close  \
0  2001-01-02 00:00:00    SPY  132.00000  132.15625  127.56250  128.812500   
1  2001-01-03 00:00:00    SPY  128.31250  136.00000  127.65625  135.000000   
2  2001-01-04 00:00:00    SPY  134.93750  135.46875  133.00000  133.546875   
3  2001-01-05 00:00:00    SPY  133.46875  133.62500  129.18750  129.187500   
4  2001-01-08 00:00:00    SPY  129.87500  130.18750  127.68750  130.187500   

   adj_close    volume  
0  81.997665   8737500  
1  85.936356  19431600  
2  85.011391   9219000  
3  82.236374  12911400  
4  82.872902   6625300  


In [15]:
df

,date,ticker,open,high,low,close,adj_close,volume
0,2001-01-02 00:00:00,SPY,132.000000,132.156250,127.562500,128.812500,81.997665,8737500
1,2001-01-03 00:00:00,SPY,128.312500,136.000000,127.656250,135.000000,85.936356,19431600
2,2001-01-04 00:00:00,SPY,134.937500,135.468750,133.000000,133.546875,85.011391,9219000
3,2001-01-05 00:00:00,SPY,133.468750,133.625000,129.187500,129.187500,82.236374,12911400
4,2001-01-08 00:00:00,SPY,129.875000,130.187500,127.687500,130.187500,82.872902,6625300
...,...,...,...,...,...,...,...,...
12555,2025-12-15 00:00:00,SPY,685.739990,685.760010,679.250000,680.729980,678.724426,90811000
12556,2025-12-16 00:00:00,SPY,679.229980,681.080017,674.979980,678.869995,676.869934,122030600
12557,2025-12-17 00:00:00,SPY,679.890015,680.440002,671.200012,671.400024,669.421936,110625200
12558,2025-12-18 00:00:00,SPY,677.599976,680.739990,674.900024,676.469971,674.476929,108650100



## Get the yfiance fundamental data ##


In [ ]:
import pandas as pd
import yfinance as yf
import sqlite3
import datetime
import time
import os

# --- Configuration ---
DB_NAME = "sp500_data.db"
TABLE_NAME = "stock_fundamentals"
# Time to sleep between individual requests (seconds) to be polite to API
REQUEST_DELAY = 1.5 
# Time to sleep after finishing the full S&P 500 list (seconds) - 24 Hours
CYCLE_DELAY = 86400 

# The mapping of Yahoo API Keys -> Your Database Column Names
FUNDAMENTAL_MAPPING = {
    # Price & Volume
    'currentPrice': 'current_price',
    'totalCash': 'total_cash',
    'totalCashPerShare': 'total_cash_per_share',
    'ebitda': 'ebitda',
    'totalDebt': 'total_debt',
    'quickRatio': 'quick_ratio',
    'currentRatio': 'current_ratio',
    'totalRevenue': 'total_revenue',
    'debtToEquity': 'debt_to_equity',
    'revenuePerShare': 'revenue_per_share',
    'returnOnAssets': 'return_on_assets',
    'returnOnEquity': 'return_on_equity',
    'grossProfits': 'gross_profits',
    'freeCashflow': 'free_cashflow',
    'operatingCashflow': 'operating_cashflow',
    'earningsGrowth': 'earnings_growth',
    'revenueGrowth': 'revenue_growth',
    'grossMargins': 'gross_margins',
    'ebitdaMargins': 'ebitda_margins',
    'operatingMargins': 'operating_margins',
    'dividendRate': 'dividend_rate',
    'dividendYield': 'dividend_yield',
    'trailingPE': 'trail_PE',
    'forwardPE': 'forward_PE',
    'trailingEps': 'trail_Eps',
    'forwardEps': 'forward_Eps',
    'bookValue': 'book_value',
    'priceToBook': 'price_2_book',
    'enterpriseValue': 'company_value',
    'sharesOutstanding': 'outstanding_share',
    'heldPercentInsiders': 'insider_held_percent',
    'heldPercentInstitutions': 'institution_held_percent',
    'shortRatio': 'short_ratio',
    'beta': 'beta'
}

TICKERS = [ "SPY" ,"NVDA", "AAPL", "MSFT", "AMZN" \
           ,"GOOGL", "META", "TSLA", "BRK-B", "JPM" \
            ,"AVGO", "INTC", "WMT", "UNH", "V", "PYPL" \
            ,"MA", "HD", "BAC", "COST", "DIS" ] 

def get_sp500_tickers():
    """Fetches current S&P 500 tickers from Wikipedia."""
    try:
        url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
        tables = pd.read_html(url)
        df = tables[0]
        tickers = df['Symbol'].str.replace('.', '-', regex=False).tolist()
        return tickers
    except Exception as e:
        print(f"Error fetching S&P list: {e}")
        return []

def init_db():
    """Creates the table if it doesn't exist."""
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    
    # Dynamic schema creation based on our mapping
    # All columns will be REAL (numbers) or TEXT based on SQLite flexibility
    cols = ", ".join([f"{col} REAL" for col in FUNDAMENTAL_MAPPING.values()])
    
    create_stmt = f"""
    CREATE TABLE IF NOT EXISTS {TABLE_NAME} (
        ticker TEXT,
        scraped_date TEXT,
        {cols},
        PRIMARY KEY (ticker, scraped_date)
    )
    """
    cursor.execute(create_stmt)
    conn.commit()
    conn.close()

def already_scraped(ticker, date_str):
    """Checks if we already have data for this ticker on this date."""
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    cursor.execute(f"SELECT 1 FROM {TABLE_NAME} WHERE ticker = ? AND scraped_date = ?", (ticker, date_str))
    exists = cursor.fetchone() is not None
    conn.close()
    return exists

def run_daily_cycle():
    """Runs one full pass of the S&P 500."""
    tickers = get_sp500_tickers()
    if not tickers:
        print("Could not retrieve tickers. Retrying later.")
        return

    today_str = datetime.date.today().strftime("%Y-%m-%d")
    print(f"--- Starting scrape cycle for {today_str} ({len(tickers)} tickers) ---")

    for ticker in tickers:
        # 1. Check for duplicates
        if already_scraped(ticker, today_str):
            print(f"[{ticker}] Already scraped for today. Skipping.")
            continue

        try:
            # 2. Fetch Data
            # Using Ticker(ticker) is faster than yf.download for fundamentals
            stock = yf.Ticker(ticker)
            info = stock.info
            
            # 3. Build Row
            row_data = {
                'ticker': ticker,
                'scraped_date': today_str
            }
            
            # Map keys and handle NULLs
            for api_key, db_col in FUNDAMENTAL_MAPPING.items():
                # .get(key, None) ensures that if the key is missing, 
                # Python uses None, which becomes NULL in SQLite.
                val = info.get(api_key, None)
                row_data[db_col] = val

            # 4. Save to DB
            df = pd.DataFrame([row_data])
            conn = sqlite3.connect(DB_NAME)
            df.to_sql(TABLE_NAME, conn, if_exists="append", index=False)
            conn.close()
            
            print(f"[{ticker}] Success.")
            
            # Sleep to be polite
            time.sleep(REQUEST_DELAY)

        except Exception as e:
            print(f"[{ticker}] Error: {e}")
            # If there is an error, we continue to the next ticker
            continue

    print(f"--- Cycle for {today_str} complete. ---")

def main():
    # Setup database once on start
    init_db()
    
    print("Scraper started. Press Ctrl+C to stop.")
    
    while True:
        try:
            run_daily_cycle()
            
            print(f"Sleeping for {CYCLE_DELAY/3600} hours...")
            time.sleep(CYCLE_DELAY)
            
        except KeyboardInterrupt:
            print("\nStopping script manually.")
            break
        except Exception as e:
            print(f"Critical Error in main loop: {e}")
            print("Retrying in 1 minute...")
            time.sleep(60)

if __name__ == "__main__":
    main()

Scraper started. Press Ctrl+C to stop.
Error fetching S&P list: HTTP Error 403: Forbidden
Could not retrieve tickers. Retrying later.
Sleeping for 24.0 hours...


In [ ]:
import pandas as pd
import yfinance as yf
import sqlite3
import datetime
import time
import os

class StockScraper:
    def __init__(self, ticker, db_name, db_folder):
        """
        Initializes the scraper and sets up the database connection.
        """
        # 1. Setup Database Path
        if not os.path.exists(db_folder):
            os.makedirs(db_folder)
        
        self.db_path = os.path.join(db_folder, db_name)
        self.table_name = ticker
        self.request_delay = 1.5
        
        # 2. Define Mapping (API Key -> DB Column)
        self.mapping = {
            'last_price': 'last',
            'bid': 'bid',
            'ask': 'ask',
            'bidSize': 'bid_size',
            'askSize': 'ask_size',
            'currentPrice': 'current_price',
            'targetHighPrice': 'target_high_price',
            'targetLowPrice': 'target_low_price',
            'targetMeanPrice': 'target_mean_price',
            'targetMedianPrice': 'target_median_price',
            'recommendationKey': 'recommendation_key', # This is TEXT (e.g., "buy")
            'numberOfAnalystOpinions': 'number_of_analyst_opinions',
            'lastDividendValue': 'last_dividend_value',
            'lastDividendDate': 'last_dividend_date',
            'totalCash': 'total_cash',
            'totalCashPerShare': 'total_cash_per_share',
            'ebitda': 'ebitda',
            'totalDebt': 'total_debt',
            'quickRatio': 'quick_ratio',
            'currentRatio': 'current_ratio',
            'totalRevenue': 'total_revenue',
            'debtToEquity': 'debt_to_equity',
            'revenuePerShare': 'revenue_per_share',
            'returnOnAssets': 'return_on_assets',
            'returnOnEquity': 'return_on_equity',
            'grossProfits': 'gross_profits',
            'freeCashflow': 'free_cashflow',
            'operatingCashflow': 'operating_cashflow',
            'earningsGrowth': 'earnings_growth',
            'revenueGrowth': 'revenue_growth',
            'grossMargins': 'gross_margins',
            'ebitdaMargins': 'ebitda_margins',
            'operatingMargins': 'operating_margins',
            'regularMarketChange': 'regular_market_change',
            'regularMarketChangePercent': 'regular_market_change_percent',
            'regularMarketPrice': 'regular_market_price',
            'postMarketChangePercent': 'post_market_change_percent',
            'postMarketPrice': 'post_market_price',
            'postMarketChange': 'post_market_change',
            'averageDailyVolume3Month': 'average_daily_volume_3_month',
            'open': 'open_price',
            'dayLow': 'day_low',
            'dayHigh': 'day_high',
            'regularMarketPreviousClose': 'regular_market_previous_close',
            'regularMarketOpen': 'regular_market_open',
            'regularMarketDayLow': 'regular_market_day_low',
            'regularMarketDayHigh': 'regular_market_day_high',
            'dividendRate': 'dividend_rate',
            'dividendYield': 'dividend_yield',
            'trailingPE': 'trail_PE',
            'forwardPE': 'forward_PE',
            'trailingEps': 'trail_Eps',
            'forwardEps': 'forward_Eps',
            'previous_close': 'prev_close',
            'volume': 'volume',
            'regularMarketVolume': 'reg_market_volume',
            'averageVolume': 'avg_volume',
            'averageVolume10days': 'ave_10d_volume',
            'currency': 'currency', # This is TEXT (e.g., "USD")
            'exchange': 'exchange', # This is TEXT (e.g., "NMS")
            'profitMargins': 'profit_margin',
            'floatShares': 'float_shares',
            'enterpriseValue': 'company_value',
            'sharesOutstanding': 'outstanding_share',
            'sharesShort': 'share_short',
            'sharesPercentSharesOut': 'share_percent_shares_out',
            'heldPercentInsiders': 'insider_held_percent',
            'heldPercentInstitutions': 'institution_held_percent',
            'shortRatio': 'short_ratio',
            'shortPercentOfFloat': 'short_percent_of_float',
            'impliedSharesOutstanding': 'implied_shares_outstanding',
            'bookValue': 'book_value',
            'priceToBook': 'price_2_book'
        }
        
        # 3. Initialize DB Schema immediately
        self._init_db()

    def _get_conn(self):
        """Helper to get a database connection."""
        return sqlite3.connect(self.db_path)

    def _init_db(self):
        """Creates the table if it doesn't exist."""
        conn = self._get_conn()
        cursor = conn.cursor()
        
        # --- CHANGE 1: DYNAMIC TYPING ---
        # Instead of f"{col} REAL", we just use f"{col}".
        # This tells SQLite to accept ANY data type (Text, Integer, Float, Null)
        # which is perfect for mixed data (prices, currency codes, ratings).
        cols = ", ".join([f"{col}" for col in self.mapping.values()])
        
        # --- CHANGE 2: ADDED TIMESTAMP COLUMN ---
        create_stmt = f"""
        CREATE TABLE IF NOT EXISTS {self.table_name} (
            ticker TEXT,
            scraped_date TEXT,
            timestamp TEXT,
            {cols},
            PRIMARY KEY (ticker, scraped_date)
        )
        """
        cursor.execute(create_stmt)
        conn.commit()
        conn.close()

    def _is_already_scraped(self, ticker, date_str):
        """Checks if data exists for this ticker/date combo."""
        conn = self._get_conn()
        cursor = conn.cursor()
        cursor.execute(
            f"SELECT 1 FROM {self.table_name} WHERE ticker = ? AND scraped_date = ?", 
            (ticker, date_str)
        )
        exists = cursor.fetchone() is not None
        conn.close()
        return exists

    def _fetch_sp500_list(self):
        """Internal helper to get S&P 500 tickers from Wiki."""
        try:
            url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
            tables = pd.read_html(url)
            # Replace dots with dashes (BRK.B -> BRK-B)
            return tables[0]['Symbol'].str.replace('.', '-', regex=False).tolist()
        except Exception as e:
            print(f"Error fetching S&P list: {e}")
            return []

    # ==========================================
    # Public Methods
    # ==========================================

    def scrape_single(self, ticker, force_update=False):
        """
        Scrapes a single ticker. 
        Returns True if successful, False if failed or skipped.
        """
        # Get current date and exact timestamp
        now = datetime.datetime.now()
        today_str = now.strftime("%Y-%m-%d")
        timestamp_str = now.strftime("%Y-%m-%d %H:%M:%S")
        
        if not force_update and self._is_already_scraped(ticker, today_str):
            print(f"[{ticker}] Already scraped for {today_str}. Skipping.")
            return False

        try:
            stock = yf.Ticker(ticker)
            info = stock.info
            
            # --- CHANGE 3: SAVE TIMESTAMP ---
            row_data = {
                'ticker': ticker, 
                'scraped_date': today_str,
                'timestamp': timestamp_str
            }
            
            # Map data
            for api_key, db_col in self.mapping.items():
                row_data[db_col] = info.get(api_key, None)

            # Save
            conn = self._get_conn()
            df = pd.DataFrame([row_data])
            df.to_sql(self.table_name, conn, if_exists="append", index=False)
            conn.close()
            
            print(f"[{ticker}] Successfully saved at {timestamp_str}.")
            time.sleep(self.request_delay) # Be polite
            return True

        except Exception as e:
            print(f"[{ticker}] Error: {e}")
            return False

    def scrape_custom_list(self, ticker_list):
        """Scrapes a specific list of tickers (e.g., your custom portfolio)."""
        print(f"--- Starting Custom Scrape ({len(ticker_list)} tickers) ---")
        for ticker in ticker_list:
            self.scrape_single(ticker)
        print("--- Custom Scrape Complete ---")

    def scrape_sp500(self):
        """Scrapes the entire S&P 500 index."""
        tickers = self._fetch_sp500_list()
        print(f"--- Starting S&P 500 Scrape ({len(tickers)} tickers) ---")
        for ticker in tickers:
            self.scrape_single(ticker)
        print("--- S&P 500 Scrape Complete ---")

# ==========================================
# Execution Example
# ==========================================

if __name__ == "__main__":
    # 1. Initialize the Scraper
    scraper = StockScraper(db_name="my_portfolio.db", db_folder="data")

    # 2. Your Custom List
    MY_TICKERS = [ 
        "SPY", "NVDA", "AAPL", "MSFT", "AMZN",
        "GOOGL", "META", "TSLA", "BRK-B", "JPM",
        "AVGO", "INTC", "WMT", "UNH", "V", "PYPL",
        "MA", "HD", "BAC", "COST", "DIS" 
    ]

    # Run the custom list scrape
    scraper.scrape_custom_list(MY_TICKERS)

## This is to check for the url connection 

In [6]:
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
try:
    tables = pd.read_html(url)
    df = tables[0]

except Exception as e:
    print(f"Error")

Error


In [7]:
tables = pd.read_html(url)

HTTPError: HTTP Error 403: Forbidden